In [ ]:
import my_library.common_analysis_tools as tools
import my_library.kinematic_cuts as kcuts
import my_library.constants as constants
import ROOT
import numpy as np

In [ ]:
ROOT.gStyle.SetOptStat(0)

In [ ]:
# TODO: move to common analysis toolS
@ROOT.Numba.Declare(['float', 'float', 'float'], 'float')
def get_theta(px, py, pz):
    return np.degrees(np.arctan2(np.sqrt(px**2 + py**2), pz))

@ROOT.Numba.Declare(['float', 'float'], 'float')
def get_phi(px, py):
    return np.degrees(np.arctan2(py, px))


In [ ]:
def make_kinematics_hists(particle, df, pxmin=-2.0, pxmax=2.0, pymin=-2.0, pymax=2.0, pzmin=-2.0, pzmax=2.0, pmin=-2.0, pmax=2.0, thetamin=0.0, thetamax=180.0, bin_size_p=0.01, bin_size_theta=1.0):
        n_bins_px = int((pxmax - pxmin) / bin_size_p)
        n_bins_py = int((pymax - pymin) / bin_size_p)
        n_bins_pz = int((pzmax - pzmin) / bin_size_p)
        n_bins_p = int((pmax - pmin) / bin_size_p)
        n_bins_theta = int((thetamax - thetamin) / bin_size_theta)

        hist_px = df.Histo1D((f'{particle}_px', f'{particle}_px', n_bins_px, pxmin, pxmax), f'{particle}_px')
        hist_py = df.Histo1D((f'{particle}_py', f'{particle}_py', n_bins_py, pymin, pymax), f'{particle}_py')
        hist_pz = df.Histo1D((f'{particle}_pz', f'{particle}_pz', n_bins_pz, pzmin, pzmax), f'{particle}_pz')
        hist_p = df.Define('p', f'sqrt({particle}_px*p_px + {particle}_py*p_py + {particle}_pz*p_pz)').Histo1D((f'{particle}_p', f'{particle}_p', n_bins_p, pmin, pmax), 'p')
        hist_theta = df.Define('theta', f'Numba::get_theta({particle}_px, {particle}_py, {particle}_pz)').Histo1D((f'{particle}_theta', f'{particle}_theta', n_bins_theta, thetamin, thetamax), 'theta')
        hist_phi = df.Define('phi', f'Numba::get_phi({particle}_px, {particle}_py)').Histo1D((f'{particle}_phi', f'{particle}_phi', 370, -185.0, 185.0), 'phi')
        pvtheta_title = "P vs #Theta for {}".format(particle)
        hist_p_vs_theta = df.Define('theta', f'Numba::get_theta({particle}_px, {particle}_py, {particle}_pz)').Define('p', f'sqrt({particle}_px*p_px + {particle}_py*p_py + {particle}_pz*p_pz)').Histo2D((f'{particle}_pvtheta', pvtheta_title, n_bins_p, pmin, pmax, n_bins_theta, thetamin, thetamax), 'p', 'theta')
        return hist_px, hist_py, hist_pz, hist_p, hist_theta, hist_phi, hist_p_vs_theta


def get_kinematics(channel: str, particle: str, df: ROOT.RDataFrame, pxmin=-2.0, pxmax=2.0, pymin=-2.0, pymax=2.0, pzmin=-2.0, pzmax=2.0, pmin=-2.0, pmax=2.0, thetamin=0.0, thetamax=180.0, bin_size_p=0.01, bin_size_theta=1.0):
    if particle not in ['proton', 'kaon', 'pion', 'kshort_pi+', 'kshort_pi-']:
        print(f'Particle {particle}_{channel} not supported')
        raise ValueError(f'Acceptable particles are proton, kaon, pion, kshort_pi+, kshort_pi-')
    if channel not in ['pipkmks', 'pimkpks']:
        print(f'Channel {channel} not supported')
        raise ValueError(f'Acceptable channels are pipkmks, pimkpks')
    
    if channel == 'pipkmks':
        kaon = 'km'
        pion = 'pip1'
        ks_pip = 'pip2'
        ks_pim = 'pim'
    elif channel == 'pimkpks':
        kaon = 'kp'
        pion = 'pim1'
        ks_pip = 'pip'
        ks_pim = 'pim2'

    if particle == 'proton':
        return make_kinematics_hists('p', df, pxmin=pxmin, pxmax=pxmax, pymin=pymin, pymax=pymax, pzmin=pzmin, pzmax=pzmax, pmin=pmin, pmax=pmax, thetamin=thetamin, thetamax=thetamax, bin_size_p=bin_size_p, bin_size_theta=bin_size_theta)
    elif particle == 'kaon':
        return make_kinematics_hists(kaon, df, pxmin=pxmin, pxmax=pxmax, pymin=pymin, pymax=pymax, pzmin=pzmin, pzmax=pzmax, pmin=pmin, pmax=pmax, thetamin=thetamin, thetamax=thetamax, bin_size_p=bin_size_p, bin_size_theta=bin_size_theta)
    elif particle == 'pion':
        return make_kinematics_hists(pion, df, pxmin=pxmin, pxmax=pxmax, pymin=pymin, pymax=pymax, pzmin=pzmin, pzmax=pzmax, pmin=pmin, pmax=pmax, thetamin=thetamin, thetamax=thetamax, bin_size_p=bin_size_p, bin_size_theta=bin_size_theta)
    elif particle == 'kshort_pi+':
        return make_kinematics_hists(ks_pip, df, pxmin=pxmin, pxmax=pxmax, pymin=pymin, pymax=pymax, pzmin=pzmin, pzmax=pzmax, pmin=pmin, pmax=pmax, thetamin=thetamin, thetamax=thetamax, bin_size_p=bin_size_p, bin_size_theta=bin_size_theta)
    elif particle == 'kshort_pi-':
        return make_kinematics_hists(ks_pim, df, pxmin=pxmin, pxmax=pxmax, pymin=pymin, pymax=pymax, pzmin=pzmin, pzmax=pzmax, pmin=pmin, pmax=pmax, thetamin=thetamin, thetamax=thetamax, bin_size_p=bin_size_p, bin_size_theta=bin_size_theta)
    

In [ ]:
run_period = 'spring'
data_type = 'signal'
df_pipkmks = tools.get_dataframe('pipkmks', run_period, data_type)
df_pimkpks = tools.get_dataframe('pimkpks', run_period, data_type)

In [ ]:
df_pipkmks = df_pipkmks.Filter(kcuts.KSTAR_ALL_CUT_PIPKMKS).Filter('pipkmks_m > 1.24 && pipkmks_m < 1.35')
df_pimkpks = df_pimkpks.Filter(kcuts.KSTAR_ALL_CUT_PIMKPKS).Filter('pimkpks_m > 1.24 && pimkpks_m < 1.35')

In [ ]:
print(df_pipkmks.GetColumnNames())

### Proton Kinematics

In [ ]:
c = ROOT.TCanvas("c", "c", 1000, 1000)
c.Divide(4, 2)

proton_hists_pipkmks = get_kinematics('pipkmks', 'proton', df_pipkmks, pmin=0.0, pzmin=0.0, thetamin=0.0, thetamax=80)
proton_hists_pimkpks = get_kinematics('pimkpks', 'proton', df_pimkpks, pmin=0.0, pzmin=0.0, thetamin=0.0, thetamax=80)
legends = []
for i, hist_pipkmks in enumerate(proton_hists_pipkmks):
    c.cd(i+1)
    hist_pipkmks = hist_pipkmks.GetValue()
    hist_pimkpks = proton_hists_pimkpks[i].GetValue()
    quantity = hist_pimkpks.GetName().split('_')[1]
    if(quantity != 'pvtheta'):
        hist_pimkpks.SetTitle(quantity + ' for p')
        hist_pimkpks.SetLineColor(ROOT.TColor.GetColor(constants.COLORBLIND_HEX_DICT['red']))
        hist_pipkmks.SetLineColor(ROOT.TColor.GetColor(constants.COLORBLIND_HEX_DICT['blue']))
        # hist_pipkmks.Scale(1.0/hist_pipkmks.Integral())
        # hist_pimkpks.Scale(1.0/hist_pimkpks.Integral())
        legend = ROOT.TLegend(0.7, 0.8, 0.95, 0.95)
        label_pipkmks = quantity + ' for #pi^{+}K^{-}K_{s}'
        label_pimkpks = quantity + ' for #pi^{-}K^{+}K_{s}'
        legend.AddEntry(hist_pipkmks, label_pipkmks, 'l')
        legend.AddEntry(hist_pimkpks, label_pimkpks, 'l')
        legends.append(legend)
        hist_pimkpks.Draw()
        hist_pipkmks.Draw('same')
        legends[i].Draw()
    else: 
        hist_pipkmks.SetTitle('P vs #Theta for p - #pi^{+}K^{-}K_{s}')
        hist_pimkpks.SetTitle('P vs #Theta for p - #pi^{-}K^{+}K_{s}')
        hist_pimkpks.Draw('colz')
        c.cd(i+2)
        hist_pipkmks.Draw('colz')
c.Draw()

### Kaon Kinematics

In [ ]:
kaon_hists_pipkmks = get_kinematics('pipkmks', 'kaon', df_pipkmks, pzmin=0.0, pzmax=6.0, pmin=0.5, pmax=2.2, thetamax=20)
kaon_hists_pimkpks = get_kinematics('pimkpks', 'kaon', df_pimkpks, pzmin=0.0, pzmax=6.0, pmin=0.5, pmax=2.2, thetamax=20)
legends = []
for i, hist_pipkmks in enumerate(kaon_hists_pipkmks):
    c.cd(i+1)
    hist_pipkmks = hist_pipkmks.GetValue()
    hist_pimkpks = kaon_hists_pimkpks[i].GetValue()
    quantity = hist_pimkpks.GetName().split('_')[1]
    print(quantity)
    if(quantity != 'pvtheta'):
        hist_pimkpks.SetTitle(quantity + ' for kaon')
        hist_pimkpks.SetLineColor(ROOT.TColor.GetColor(constants.COLORBLIND_HEX_DICT['red']))
        hist_pipkmks.SetLineColor(ROOT.TColor.GetColor(constants.COLORBLIND_HEX_DICT['blue']))
        # hist_pipkmks.Scale(1.0/hist_pipkmks.Integral())
        # hist_pimkpks.Scale(1.0/hist_pimkpks.Integral())
        legend = ROOT.TLegend(0.7, 0.8, 0.95, 0.95)
        label_pipkmks = quantity + ' for #pi^{+}K^{-}K_{s}'
        label_pimkpks = quantity + ' for #pi^{-}K^{+}K_{s}'
        legend.AddEntry(hist_pipkmks, label_pipkmks, 'l')
        legend.AddEntry(hist_pimkpks, label_pimkpks, 'l')
        legends.append(legend)
        hist_pimkpks.Draw()
        hist_pipkmks.Draw('same')
        legends[i].Draw()
    else: 
        hist_pipkmks.SetTitle('P vs #Theta for kaon - #pi^{+}K^{-}K_{s}')
        hist_pimkpks.SetTitle('P vs #Theta for kaon - #pi^{-}K^{+}K_{s}')
        hist_pimkpks.Draw('colz')
        c.cd(i+2)
        hist_pipkmks.Draw('colz')
c.Draw()

### Pion Kinematics

In [ ]:
pion_hists_pipkmks = get_kinematics('pipkmks', 'pion', df_pipkmks, pzmin=0.0, pzmax=3.0, pmin=0.0, pmax=1.5, thetamax=60)
pion_hists_pimkpks = get_kinematics('pimkpks', 'pion', df_pimkpks, pzmin=0.0, pzmax=3.0, pmin=0.0, pmax=1.5, thetamax=60)
legends = []
for i, hist_pipkmks in enumerate(pion_hists_pipkmks):
    c.cd(i+1)
    hist_pipkmks = hist_pipkmks.GetValue()
    hist_pimkpks = pion_hists_pimkpks[i].GetValue()
    quantity = hist_pimkpks.GetName().split('_')[1]
    if(quantity != 'pvtheta'):
        hist_pimkpks.SetTitle(quantity + ' for pion')
        hist_pimkpks.SetLineColor(ROOT.TColor.GetColor(constants.COLORBLIND_HEX_DICT['red']))
        hist_pipkmks.SetLineColor(ROOT.TColor.GetColor(constants.COLORBLIND_HEX_DICT['blue']))
        # hist_pipkmks.Scale(1.0/hist_pipkmks.Integral())
        # hist_pimkpks.Scale(1.0/hist_pimkpks.Integral())
        legend = ROOT.TLegend(0.7, 0.8, 0.95, 0.95)
        label_pipkmks = quantity + ' for #pi^{+}K^{-}K_{s}'
        label_pimkpks = quantity + ' for #pi^{-}K^{+}K_{s}'
        legend.AddEntry(hist_pipkmks, label_pipkmks, 'l')
        legend.AddEntry(hist_pimkpks, label_pimkpks, 'l')
        legends.append(legend)
        hist_pimkpks.Draw()
        hist_pipkmks.Draw('same')
        legends[i].Draw()
    else: 
        hist_pipkmks.SetTitle('P vs #Theta for pion - #pi^{+}K^{-}K_{s}')
        hist_pimkpks.SetTitle('P vs #Theta for pion - #pi^{-}K^{+}K_{s}')
        hist_pimkpks.Draw('colz')
        c.cd(i+2)
        hist_pipkmks.Draw('colz')
c.Draw()

### KShort Pi+ Kinematics

In [ ]:
kspip_hists_pipkmks = get_kinematics('pipkmks', 'kshort_pi+', df_pipkmks, pzmin=0.0, pzmax=4.0, pmin=0.0, thetamax=40)
kspip_hists_pimkpks = get_kinematics('pimkpks', 'kshort_pi+', df_pimkpks, pzmin=0.0, pzmax=4.0, pmin=0.0, thetamax=40)
legends = []
for i, hist_pipkmks in enumerate(kspip_hists_pipkmks):
    c.cd(i+1)
    hist_pipkmks = hist_pipkmks.GetValue()
    hist_pimkpks = kspip_hists_pimkpks[i].GetValue()
    quantity = hist_pimkpks.GetName().split('_')[1]
    if(quantity != 'pvtheta'):
        hist_pimkpks.SetTitle(quantity + ' for #pi^{+} from K_{s}')
        hist_pimkpks.SetLineColor(ROOT.TColor.GetColor(constants.COLORBLIND_HEX_DICT['red']))
        hist_pipkmks.SetLineColor(ROOT.TColor.GetColor(constants.COLORBLIND_HEX_DICT['blue']))
        # hist_pipkmks.Scale(1.0/hist_pipkmks.Integral())
        # hist_pimkpks.Scale(1.0/hist_pimkpks.Integral())
        legend = ROOT.TLegend(0.7, 0.8, 0.95, 0.95)
        label_pipkmks = quantity + ' for #pi^{+}K^{-}K_{s}'
        label_pimkpks = quantity + ' for #pi^{-}K^{+}K_{s}'
        legend.AddEntry(hist_pipkmks, label_pipkmks, 'l')
        legend.AddEntry(hist_pimkpks, label_pimkpks, 'l')
        legends.append(legend)
        hist_pimkpks.Draw()
        hist_pipkmks.Draw('same')
        legends[i].Draw()
    else: 
        hist_pipkmks.SetTitle('P vs #Theta for #pi^{+}_{K_{s}} - #pi^{+}K^{-}K_{s}')
        hist_pimkpks.SetTitle('P vs #Theta for #pi^{+}_{K_{s}} - #pi^{-}K^{+}K_{s}')
        hist_pimkpks.Draw('colz')
        c.cd(i+2)
        hist_pipkmks.Draw('colz')
c.Draw()

### KShort Pi- Kinematics

In [ ]:
kspim_hists_pipkmks = get_kinematics('pipkmks', 'kshort_pi-', df_pipkmks, pzmin=0.0, pzmax=4.0, pmin=0.0, thetamax=40)
kspim_hists_pimkpks = get_kinematics('pimkpks', 'kshort_pi-', df_pimkpks, pzmin=0.0, pzmax=4.0, pmin=0.0, thetamax=40)
legends = []
for i, hist_pipkmks in enumerate(kspim_hists_pipkmks):
    c.cd(i+1)
    hist_pipkmks = hist_pipkmks.GetValue()
    hist_pimkpks = kspim_hists_pimkpks[i].GetValue()
    quantity = hist_pimkpks.GetName().split('_')[1]
    if(quantity != 'pvtheta'):
        hist_pimkpks.SetTitle(quantity + ' for #pi^{-} from K_{s}')
        hist_pimkpks.SetLineColor(ROOT.TColor.GetColor(constants.COLORBLIND_HEX_DICT['red']))
        hist_pipkmks.SetLineColor(ROOT.TColor.GetColor(constants.COLORBLIND_HEX_DICT['blue']))
        # hist_pipkmks.Scale(1.0/hist_pipkmks.Integral())
        # hist_pimkpks.Scale(1.0/hist_pimkpks.Integral())
        legend = ROOT.TLegend(0.7, 0.8, 0.95, 0.95)
        label_pipkmks = quantity + ' for #pi^{+}K^{-}K_{s}'
        label_pimkpks = quantity + ' for #pi^{-}K^{+}K_{s}'
        legend.AddEntry(hist_pipkmks, label_pipkmks, 'l')
        legend.AddEntry(hist_pimkpks, label_pimkpks, 'l')
        legends.append(legend)
        hist_pimkpks.Draw()
        hist_pipkmks.Draw('same')
        legends[i].Draw()
    else: 
        hist_pipkmks.SetTitle('P vs #Theta for #pi^{-}_{K_{s}} - #pi^{+}K^{-}K_{s}')
        hist_pimkpks.SetTitle('P vs #Theta for #pi^{-}_{K_{s}} - #pi^{-}K^{+}K_{s}')
        hist_pimkpks.Draw('colz')
        c.cd(i+2)
        hist_pipkmks.Draw('colz')
c.Draw()